#  Few-shot распознавание сущностей

Берт и возможность тренировать few-shot ner модели есть в deep pavlov (http://docs.deeppavlov.ai/en/master/features/models/ner.html)

In [ ]:
!pip install deeppavlov

     |████████████████████████████████| 737kB 3.5MB/s 
     |████████████████████████████████| 163kB 54.9MB/s 
     |████████████████████████████████| 10.1MB 45.9MB/s 
     |████████████████████████████████| 6.7MB 43.8MB/s 
     |████████████████████████████████| 2.1MB 43.7MB/s 
     |████████████████████████████████| 25.2MB 1.5MB/s 
     |████████████████████████████████| 17.3MB 117kB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 317kB 33.0MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 2.8MB 46.4MB/s 
     |████████████████████████████████| 8.0MB 41.5MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 5.1MB 47.1MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |

In [ ]:
!pip install tensorflow==1.15.0

In [ ]:
!apt-get --yes install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.5).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [ ]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-r_x1_2nq
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-r_x1_2nq
  Running command git checkout -b feat/multi_gpu --track origin/feat/multi_gpu
  Switched to a new branch 'feat/multi_gpu'
  Branch 'feat/multi_gpu' set up to track remote branch 'feat/multi_gpu' from 'origin'.
  Created wheel for bert-dp: filename=bert_dp-1.0-cp36-none-any.whl size=23580 sha256=774401c6ce5955bac5cbc15113c466adbf72a2fd24c5b1db4170e762feb9db4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-vhfqdj5p/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp


In [ ]:
# загрузка файлов для колаба
from google.colab import files
uploaded = files.upload()

Saving pristavki.csv to pristavki.csv


[Файлы train, test и valid лежат здесь](https://drive.google.com/open?id=1u_oTAySOGMSYtoUracpIvmcd3_btPGjf)

In [ ]:
# в колабе файлы не перезаписываются, создается новый с (1,2,3..). 
# Чтобы удалить старый файл перед загрузкой нового используйте rm file
# rm .txt удаляет все txt файлы в текущей папке
#!rm *.txt

In [ ]:
import pandas as pd

Возьмем тексты с авито по категории приставки.
B - начало сущности, I - та же сущность #ILB encoding

In [ ]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

Чтобы научиться выделять названия разметим небольшое количество текстов. Для DeepPavlov разметку нужно поделить на три файла: train.txt, valid.txt, test.txt.

In [ ]:
!ls *.txt

test.txt  train.txt  valid.txt


In [ ]:
!head train.txt

PS3 O
с O
пятью O
играми O
в O
комплекте O
. O
Игры O
: O
UNCHARTED O


Попробуем теперь обучить разметчик сущностей поверх многоязычного берта.Обязательно указать путь к обучающим файлам.
а #в конце метрики теста #bpe токенизация по кусочкам

In [ ]:
import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f: #bert_mult
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

ner_model = train_model(ner_config, download=True)

2020-01-22 08:40:41.457 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [02:51<00:00, 3.87MB/s]
2020-01-22 08:43:32.653 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /root/.deeppavlov/downloads/bert_models
2020-01-22 08:43:38.572 WARNING in 'deeppavlov.dataset_readers.conll2003_reader'['conll2003_reader'] at line 96: Skip 'кO\n', splitted as ['кO']
2020-01-22 08:43:38.574 WARNING in 'deeppavlov.dataset_readers.conll2003_reader'['conll2003_reader'] at line 96: Skip 'Assasin \n', splitted as ['Assasin']
2020-01-22 08:43:38.584 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ign

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...


[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
2020-01-22 08:43:41.922 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/tag.dict]


Using TensorFlow backend.






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use standard

...

INFO:tensorflow:Restoring parameters from /content/model


2020-01-22 08:48:05.38 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 123 tokens with 5 phrases; found: 5 phrases; correct: 0.

precision:  80.00%; recall:  80.00%; FB1:  80.00

	IGR: precision:  80.00%; recall:  80.00%; F1:  80.00 5


2020-01-22 08:48:05.169 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 110 tokens with 5 phrases; found: 6 phrases; correct: 0.

precision:  50.00%; recall:  60.00%; FB1:  54.55

	IGR: precision:  50.00%; recall:  60.00%; F1:  54.55 6




{"valid": {"eval_examples_count": 5, "metrics": {"ner_f1": 80.0, "ner_token_f1": 95.4545}, "time_spent": "0:00:02"}}
{"test": {"eval_examples_count": 5, "metrics": {"ner_f1": 54.5455, "ner_token_f1": 90.9091}, "time_spent": "0:00:01"}}


2020-01-22 08:48:05.720 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/tag.dict]
2020-01-22 08:48:31.286 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/model]


INFO:tensorflow:Restoring parameters from /content/model


Посмотрим как это все размечается.

In [ ]:
ner_model(['Assasin\'s fracking freaking bloody Creed',\
           'Assasin\'s но без Creed продам',\
           'Продам assasin\'s, creed не продам'])

[[["Assasin's", 'fracking', 'freaking', 'bloody', 'Creed'],
  ["Assasin's", 'но', 'без', 'Creed', 'продам'],
  ['Продам', "assasin's", ',', 'creed', 'не', 'продам']],
 [['B-IGR', 'I-IGR', 'I-IGR', 'I-IGR', 'I-IGR'],
  ['B-IGR', 'I-IGR', 'I-IGR', 'I-IGR', 'O'],
  ['O', 'B-IGR', 'I-IGR', 'I-IGR', 'O', 'O']]]

In [ ]:
ner_model(['asasin credd игра', 'асасинс creed', 'Egor Creed',\
           'знаменитый художник Егор Creed', 'Assasin\'s крид'])

[[['asasin', 'credd', 'игра'],
  ['асасинс', 'creed'],
  ['Egor', 'Creed'],
  ['знаменитый', 'художник', 'Егор', 'Creed'],
  ["Assasin's", 'крид']],
 [['I-IGR', 'I-IGR', 'O'],
  ['O', 'B-IGR'],
  ['B-IGR', 'I-IGR'],
  ['O', 'O', 'O', 'B-IGR'],
  ['B-IGR', 'I-IGR']]]

In [ ]:
marked = []

for number, text in enumerate (data.text.values): #[:1000]
    
    # взяла вторую половину текстов, потому что из первой я брала тексты для обучения
    if number < 5000:
        continue
    
    # BERT имеет лимит на длину текста в 512 слов, возьмем даже еще меньше
    if len(text.split()) > 100:
        continue
    
    # пропускала тексты, на которые ругался берт
    try:
        pred = ner_model([text])
    except:
        continue
    
    sent, tags = pred[0][0], pred[1][0]
    # достанем только тексты с сущностями
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [ ]:
marked

[[("Assassin's", 'B-IGR'),
  ('creed', 'I-IGR'),
  ('Синдикат', 'I-IGR'),
  ('для', 'O'),
  ('PS4', 'O')],
 [('Drakengard', 'I-IGR'),
  ('3', 'O'),
  ('(', 'O'),
  ('азиатка', 'O'),
  (')', 'O'),
  ('-', 'O'),
  ('2500', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('Afrika', 'O'),
  ('(', 'O'),
  ('в', 'O'),
  ('азии', 'O'),
  ('известна', 'O'),
  ('как', 'O'),
  ('Hakuna', 'O'),
  ('Matata', 'O'),
  (')', 'O'),
  ('американка', 'O'),
  ('-', 'O'),
  ('3500', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('Blur', 'O'),
  ('-', 'O'),
  ('2000', 'O')],
 [('Uncharted', 'B-IGR'),
  ('Натан', 'I-IGR'),
  ('Дрейк', 'O'),
  ('.', 'O'),
  ('Коллекция', 'O'),
  ('-', 'O'),
  ('1400', 'O'),
  ('.', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('Ассасин', 'O'),
  ('единство', 'O'),
  ('-', 'O'),
  ('1000', 'O'),
  ('.', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('Need', 'O'),
  ('for', 'O'),
  ('speed', 'O'),
  ('rivals', 'O'),
  ('-', 'O'),
  ('700', 'O'),
  ('.', 'O

Я искала тексты для train по запросу 'creed', поэтому ожидаемо, что модель выделяет случаи, когда элемент названия асасинс крид написан латиницей, и не выделяет почти ничего из того, что написано кириллицей.
Выделяет часть игры 'Синдикат' (как в 'Assasin's creed Синдикат', потому что подобные примеры были в тренировочном сете). Кроме синдиката и откровений, кажется, не выделяет другие части на кириллице (такие как чёрный флаг и единство).

***Я размечала как игру только асасинс крид с номером или названием части***, вопреки этому выделились другие игры.

Из успехов:

*   '**Uncharted 4 Lords of fallen**'
*   '**Prince of Persia: The Forgotten Sands**'
*   '**Call of Duty Infinite Warfare** - на PS4'
*   '**Grand Theft Auto IV** для PS3'
*   '**Rise of the Tomb Raider** продам'
*   '**Wolfenstein tte new order** PS4 русские субтитры'
*   '**Rainbow Six Siege** для XBOX ONE'
*   '**Borderlands 2** для xbox 360'

Из неудач:
*   'Посетив королевство **Azad своего** брата'
*   '**Ps4 1tb** последняя ревизия, приставка в идеале'
*   '**whats app** есть'
*   '**Kaspersky Internet** Security 7'
*   '**Uncharted для Ps vita**' (ср. '**Borderlands 2** для xbox 360')
*   '**Soni Plaistahin_3** в отличном состоянии'

Почему-то характеристика издания на кириллице размечается как продолжение сущности (видимо из-за того, что в тренировочном сете были примеры, где 'синдикат' было продолжением сущности, а само название игры было на латинице):

*   '**Dying Light Полное** Издание'
*   '**The sims 3 коллекционное** издание'

При этом выделилось '**Uncharted Натан**', но не выделилось следующее за ним 'Дрейк'.

В целом попадаются приставки, размеченные как игры. Иногда начало сущности теряется, и вся она размечается как продолжение:
*   '**Mortal Kombat** 1000'
*   '**Tony Hawk** Ride с доской'

Здесь кроме игр тоже попадается мусор: '**Windoows** 7 Домашняя расширенная'.

Но не всё, что на латинице, выделяется в принципе. Не знаю, чем виндовс похоже на название игры.

Теряется номер части (типа 4, XL): '**Drakengard**' выделилось как продолжение сущности, но не  выделилось следующее за ним '3'.

Кажется, что не выделяет сущности, состоящие только из начала. Вот тут прилепился второй токен -- дефис в качестве тире (в тренировочном сете был случай, когда такой знак был в середине сущности): '**Aliens -** 1000'.